+ [Training a winning model](https://www.kaggle.com/theoviel/training-a-winning-model/notebook?scriptVersionId=42814701)

## Libraries

+ v3 pin memory True テスト

In [ ]:
!pip install ../input/torchlibrosa/torchlibrosa-0.0.5-py3-none-any.whl

In [ ]:
import sys
#sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
#sys.path.append('../input/pytorchimagemodels')
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import soundfile as sf
from pathlib import Path
from contextlib import contextmanager
from collections import Counter
from typing import Optional
import logging
from IPython.display import Audio, IFrame, display # jupyterで再生につかう

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation

import timm
import warnings 
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

train = pd.read_csv('../input/birdclef-2021/train_metadata.csv')
test = pd.read_csv('../input/birdclef-2021/test.csv')
train = train[['primary_label', 'filename']]

## CFG

In [ ]:
TARGETS = [
        'acafly', 'acowoo', 'aldfly', 'ameavo', 'amecro',
        'amegfi', 'amekes', 'amepip', 'amered', 'amerob',
        'amewig', 'amtspa', 'andsol1', 'annhum', 'astfly',
        'azaspi1', 'babwar', 'baleag', 'balori', 'banana',
        'banswa', 'banwre1', 'barant1', 'barswa', 'batpig1',
        'bawswa1', 'bawwar', 'baywre1', 'bbwduc', 'bcnher',
        'belkin1', 'belvir', 'bewwre', 'bkbmag1', 'bkbplo',
        'bkbwar', 'bkcchi', 'bkhgro', 'bkmtou1', 'bknsti', 'blbgra1',
        'blbthr1', 'blcjay1', 'blctan1', 'blhpar1', 'blkpho',
        'blsspa1', 'blugrb1', 'blujay', 'bncfly', 'bnhcow', 'bobfly1',
        'bongul', 'botgra', 'brbmot1', 'brbsol1', 'brcvir1', 'brebla',
        'brncre', 'brnjay', 'brnthr', 'brratt1', 'brwhaw', 'brwpar1',
        'btbwar', 'btnwar', 'btywar', 'bucmot2', 'buggna', 'bugtan',
        'buhvir', 'bulori', 'burwar1', 'bushti', 'butsal1', 'buwtea',
        'cacgoo1', 'cacwre', 'calqua', 'caltow', 'cangoo', 'canwar',
        'carchi', 'carwre', 'casfin', 'caskin', 'caster1', 'casvir',
        'categr', 'ccbfin', 'cedwax', 'chbant1', 'chbchi', 'chbwre1',
        'chcant2', 'chispa', 'chswar', 'cinfly2', 'clanut', 'clcrob',
        'cliswa', 'cobtan1', 'cocwoo1', 'cogdov', 'colcha1', 'coltro1',
        'comgol', 'comgra', 'comloo', 'commer', 'compau', 'compot1',
        'comrav', 'comyel', 'coohaw', 'cotfly1', 'cowscj1', 'cregua1',
        'creoro1', 'crfpar', 'cubthr', 'daejun', 'dowwoo', 'ducfly', 'dusfly',
        'easblu', 'easkin', 'easmea', 'easpho', 'eastow', 'eawpew', 'eletro',
        'eucdov', 'eursta', 'fepowl', 'fiespa', 'flrtan1', 'foxspa', 'gadwal',
        'gamqua', 'gartro1', 'gbbgul', 'gbwwre1', 'gcrwar', 'gilwoo',
        'gnttow', 'gnwtea', 'gocfly1', 'gockin', 'gocspa', 'goftyr1',
        'gohque1', 'goowoo1', 'grasal1', 'grbani', 'grbher3', 'grcfly',
        'greegr', 'grekis', 'grepew', 'grethr1', 'gretin1', 'greyel',
        'grhcha1', 'grhowl', 'grnher', 'grnjay', 'grtgra', 'grycat',
        'gryhaw2', 'gwfgoo', 'haiwoo', 'heptan', 'hergul', 'herthr',
        'herwar', 'higmot1', 'hofwoo1', 'houfin', 'houspa', 'houwre',
        'hutvir', 'incdov', 'indbun', 'kebtou1', 'killde', 'labwoo', 'larspa',
        'laufal1', 'laugul', 'lazbun', 'leafly', 'leasan', 'lesgol', 'lesgre1',
        'lesvio1', 'linspa', 'linwoo1', 'littin1', 'lobdow', 'lobgna5', 'logshr',
        'lotduc', 'lotman1', 'lucwar', 'macwar', 'magwar', 'mallar3', 'marwre',
        'mastro1', 'meapar', 'melbla1', 'monoro1', 'mouchi', 'moudov', 'mouela1',
        'mouqua', 'mouwar', 'mutswa', 'naswar', 'norcar', 'norfli', 'normoc', 'norpar',
        'norsho', 'norwat', 'nrwswa', 'nutwoo', 'oaktit', 'obnthr1', 'ocbfly1',
        'oliwoo1', 'olsfly', 'orbeup1', 'orbspa1', 'orcpar', 'orcwar', 'orfpar',
        'osprey', 'ovenbi1', 'pabspi1', 'paltan1', 'palwar', 'pasfly', 'pavpig2',
        'phivir', 'pibgre', 'pilwoo', 'pinsis', 'pirfly1', 'plawre1', 'plaxen1',
        'plsvir', 'plupig2', 'prowar', 'purfin', 'purgal2', 'putfru1', 'pygnut',
        'rawwre1', 'rcatan1', 'rebnut', 'rebsap', 'rebwoo', 'redcro', 'reevir1',
        'rehbar1', 'relpar', 'reshaw', 'rethaw', 'rewbla', 'ribgul', 'rinkin1',
        'roahaw', 'robgro', 'rocpig', 'rotbec', 'royter1', 'rthhum', 'rtlhum',
        'ruboro1', 'rubpep1', 'rubrob', 'rubwre1', 'ruckin', 'rucspa1', 'rucwar',
        'rucwar1', 'rudpig', 'rudtur', 'rufhum', 'rugdov', 'rumfly1', 'runwre1',
        'rutjac1', 'saffin', 'sancra', 'sander', 'savspa', 'saypho', 'scamac1',
        'scatan', 'scbwre1', 'scptyr1', 'scrtan1', 'semplo', 'shicow', 'sibtan2',
        'sinwre1', 'sltred', 'smbani', 'snogoo', 'sobtyr1', 'socfly1', 'solsan',
        'sonspa', 'soulap1', 'sposan', 'spotow', 'spvear1', 'squcuc1', 'stbori',
        'stejay', 'sthant1', 'sthwoo1', 'strcuc1', 'strfly1', 'strsal1', 'stvhum2',
        'subfly', 'sumtan', 'swaspa', 'swathr', 'tenwar', 'thbeup1', 'thbkin',
        'thswar1', 'towsol', 'treswa', 'trogna1', 'trokin', 'tromoc', 'tropar',
        'tropew1', 'tuftit', 'tunswa', 'veery', 'verdin', 'vigswa', 'warvir',
        'wbwwre1', 'webwoo1', 'wegspa1', 'wesant1', 'wesblu', 'weskin', 'wesmea',
        'westan', 'wewpew', 'whbman1', 'whbnut', 'whcpar', 'whcsee1', 'whcspa',
        'whevir', 'whfpar1', 'whimbr', 'whiwre1', 'whtdov', 'whtspa', 'whwbec1',
        'whwdov', 'wilfly', 'willet1', 'wilsni1', 'wiltur', 'wlswar', 'wooduc',
        'woothr', 'wrenti', 'y00475', 'yebcha', 'yebela1', 'yebfly', 'yebori1',
        'yebsap', 'yebsee1', 'yefgra1', 'yegvir', 'yehbla', 'yehcar1', 'yelgro',
        'yelwar', 'yeofly1', 'yerwar', 'yeteup1', 'yetvir']

In [ ]:
class CFG:
    seed = 29
    target_col = 'primary_label'
    train_datadir = Path("../input/birdclef-2021/train_short_audio")
    period = 5
    img_size = 224
    target_size = len(TARGETS)
    # Audio cfg
    n_mels = 128
    fmin = 20
    fmax = 16000
    n_fft = 2048
    hop_length = 512
    sample_rate = 32000
    # model
    pretrained = True
    in_channels = 1

## Utils

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger

@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)

logger = get_logger("main.log")
set_seed(seed=CFG.seed)

## Data loading

In [ ]:
TAEGET_SR = 32000
TEST = (len(list(Path("../input/birdclef-2021/test_soundscapes/").glob("*.ogg"))) != 0)
TARGET_PATH = None
if TEST:
    print(f'TEST mode')
    DATADIR = Path("../input/birdclef-2021/test_soundscapes/")
else:
    print(f'TRAIN mode')
    DATADIR = Path("../input/birdclef-2021/train_soundscapes/")
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

In [ ]:
all_audios = list(DATADIR.glob("*.ogg"))
all_audio_ids = ["_".join(audio_id.name.split("_")[:2]) for audio_id in all_audios]
submission_df = pd.DataFrame({
    "row_id": all_audio_ids
})
submission_df

## Dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df: pd.DataFrame, clip: np.ndarray,
                 waveform_transforms=None):
        self.df = df
        self.clip = clip
        self.waveform_transforms=waveform_transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        SR = 32000
        sample = self.df.loc[idx, :]
        row_id = sample.row_id
        
        end_seconds = int(sample.seconds)
        start_seconds = int(end_seconds - 5)
        
        start_index = SR * start_seconds
        end_index = SR * end_seconds
        
        y = self.clip[start_index:end_index].astype(np.float32)
        
        y = np.nan_to_num(y)

        if self.waveform_transforms:
            y = self.waveform_transforms(y)

        y = np.nan_to_num(y)
        
        return y, row_id

## Torchaudio utils
https://www.kaggle.com/vladimirsydor/4-th-place-solution-inference-and-training-tips

In [ ]:
def compute_deltas(
        specgram: torch.Tensor,
        win_length: int = 5,
        mode: str = "replicate"
) -> torch.Tensor:
    r"""Compute delta coefficients of a tensor, usually a spectrogram:

    .. math::
       d_t = \frac{\sum_{n=1}^{\text{N}} n (c_{t+n} - c_{t-n})}{2 \sum_{n=1}^{\text{N}} n^2}

    where :math:`d_t` is the deltas at time :math:`t`,
    :math:`c_t` is the spectrogram coeffcients at time :math:`t`,
    :math:`N` is ``(win_length-1)//2``.

    Args:
        specgram (Tensor): Tensor of audio of dimension (..., freq, time)
        win_length (int, optional): The window length used for computing delta (Default: ``5``)
        mode (str, optional): Mode parameter passed to padding (Default: ``"replicate"``)

    Returns:
        Tensor: Tensor of deltas of dimension (..., freq, time)

    Example
        >>> specgram = torch.randn(1, 40, 1000)
        >>> delta = compute_deltas(specgram)
        >>> delta2 = compute_deltas(delta)
    """
    device = specgram.device
    dtype = specgram.dtype

    # pack batch
    shape = specgram.size()
    specgram = specgram.reshape(1, -1, shape[-1])

    assert win_length >= 3

    n = (win_length - 1) // 2

    # twice sum of integer squared
    denom = n * (n + 1) * (2 * n + 1) / 3

    specgram = torch.nn.functional.pad(specgram, (n, n), mode=mode)

    kernel = torch.arange(-n, n + 1, 1, device=device, dtype=dtype).repeat(specgram.shape[1], 1, 1)

    output = torch.nn.functional.conv1d(specgram, kernel, groups=specgram.shape[1]) / denom

    # unpack batch
    output = output.reshape(shape)

    return output

def make_delta(input_tensor: torch.Tensor):
    input_tensor = input_tensor.transpose(3,2)
    input_tensor = compute_deltas(input_tensor)
    input_tensor = input_tensor.transpose(3,2)
    return input_tensor

## Model

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model_name="", pretrained=False, in_channels=1,spec_aggreagation: str='repeat3'):
        super().__init__()
        
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=CFG.n_fft, hop_length=CFG.hop_length,
                                                 win_length=CFG.n_fft, window="hann", center=True, pad_mode="reflect",
                                                 freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=CFG.sample_rate, n_fft=CFG.n_fft,
                                                 n_mels=CFG.n_mels, fmin=CFG.fmin, fmax=CFG.fmax, ref=1.0, amin=1e-10, top_db=None,
                                                 freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2,
                                               freq_drop_width=8, freq_stripes_num=2)
        self.bn0 = nn.BatchNorm2d(CFG.n_mels)
        
        self.spec_aggreagation = spec_aggreagation

        self.model = timm.create_model(model_name, pretrained=pretrained,in_chans=in_channels)
        
        if 'rexnet' in model_name:
            n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Linear(n_features, CFG.target_size)
        elif ('efficientnet' in model_name) or ('densenet' in model_name):
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, CFG.target_size)
        elif ('resnet' in model_name) or ('resnext' in model_name)or ('resnest' in model_name):
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, CFG.target_size)
        else:
            raise NotImplementedError
            

    def forward(self, input):
        """
        Input: (batch_size, data_length)
        """
        x = self.spectrogram_extractor(input)# output:(batch_size, 1(channel), time_steps, freq_bins)
        x = self.logmel_extractor(x)# output:(batch_size, 1(channel), time_steps, mel_bins)
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        if self.training:
            x = self.spec_augmenter(x)
        # 1channel => 3channel
        if self.spec_aggreagation == 'repeat3':
            x = torch.cat([x,x,x], dim=1)
        elif self.spec_aggreagation == 'deltas':
            delta_1 = make_delta(x)
            delta_2 = make_delta(delta_1)
            x = torch.cat([x,delta_1,delta_2], dim=1)

        #x = x.transpose(2, 3)
        # (batch_size, channels, freq, frames)

        x = self.model(x)
        return x

In [ ]:
def prediction_for_clip(test_df: pd.DataFrame,
                        clip: np.ndarray,
                        model):
    """
    [input]test_df:audiofile一個分のdataframe
           clip:音声データ
           model:モデルのインスタンス
           threshold:
           
    [output]preds:ndarray(120,397)
    
    """
    model.eval()
    preds = np.empty((0, len(TARGETS)))
    dataset = TestDataset(df=test_df,
                          clip=clip,
                          waveform_transforms=None)
    loader = DataLoader(dataset, batch_size=120, shuffle=False)
    
    for image, row_id in loader:
        row_id = row_id[0]
        image = image.to(device)
        with torch.no_grad():
            y_pred = model(image)
            preds = np.concatenate([preds, torch.sigmoid(y_pred).cpu().numpy()])
        
    return preds

In [ ]:
def get_model(model_name, pretrained=False, in_channels=3, spec_aggreagation='repeat3'):
    # if 'efficientnet' in model_name:
    #     model = CustomEfficientNet(model_name=model_name, pretrained=pretrained, in_channels=in_channels)
    # elif 'rexnet' in model_name:
    #     model = CustomRexNet(model_name=model_name, pretrained=pretrained, in_channels=in_channels)
    # else:
    #     raise NotImplementedError
    model = CustomModel(model_name=model_name,
                        pretrained=pretrained,
                        in_channels=in_channels,
                        spec_aggreagation=spec_aggreagation)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    return model

def load_model_weights(model, weights):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state_dict = torch.load(weights, map_location=device)
    model.load_state_dict(state_dict["model"])

Inferene with voting

# ppNo3

In [ ]:
def make_location_list(target_list):
    location_list = np.zeros(len(TARGETS))
    for i in range(len(TARGETS)):
        if TARGETS[i] in target_list:
            location_list[i] = 1
    return location_list

col_name_list = pd.read_csv("../input/bird2location-list/col_list.csv")["bird"].values.tolist()
cor_name_list = pd.read_csv("../input/bird2location-list/cor_list.csv")["bird"].values.tolist()
sne_name_list = pd.read_csv("../input/bird2location-list/sne_list.csv")["bird"].values.tolist()
ssw_name_list = pd.read_csv("../input/bird2location-list/ssw_list.csv")["bird"].values.tolist()

COL_list = make_location_list(col_name_list)
COR_list = make_location_list(cor_name_list)
SNE_list = make_location_list(sne_name_list)
SSW_list = make_location_list(ssw_name_list)

# ppNo2,4,6,7

In [ ]:
def get_adjust_score(score, optim_thresh):
    # ppNo6
    tmp = np.copy(score)
    for i in range(len(tmp)):
        tmp[i,:] = tmp[i,:] - optim_thresh# ppNo4
    sum_score = np.sum(tmp>0, axis=0)#(397,)
    adjust_score = np.zeros(397)
    #print(max_score.shape, adjust_score.shape)
    adjust_score[sum_score>10] = 0.1

    return adjust_score

def post_process_site_12(preds, target_list, optim_thresh, maxpreds=3):
    """
    input: preds(120, 397)
    """
    #preds = preds * (preds >= threshold)   # remove preds < threshold

    next_preds = np.concatenate([preds[1:], np.zeros((1, preds.shape[-1]))])   # pred corresponding to next window
    prev_preds = np.concatenate([np.zeros((1, preds.shape[-1])), preds[:-1]])  # pred corresponding to previous window
    n_n_preds = np.concatenate([preds[2:], np.zeros((2, preds.shape[-1]))])   # ppNo1
    p_p_preds = np.concatenate([np.zeros((2, preds.shape[-1])), preds[:-2]])  # ppNo1
    n_n_n_preds = np.concatenate([preds[3:], np.zeros((3, preds.shape[-1]))])   # ppNo1
    p_p_p_preds = np.concatenate([np.zeros((3, preds.shape[-1])), preds[:-3]])  # ppNo1
    
    score = preds + 0.5 * (next_preds + prev_preds + n_n_preds + p_p_preds + n_n_n_preds + p_p_p_preds)# ppNo1
    
    score[:, target_list==0] = 0# ppNo2
    
    # ppNo6
    adjust_score = get_adjust_score(score[:40], optim_thresh)
    adjust_score2 = get_adjust_score(score[40:80], optim_thresh)
    adjust_score3 = get_adjust_score(score[80:], optim_thresh)
    
    for i in range(len(score)):
        if i < 40:
            score[i,:] = score[i,:] - optim_thresh + adjust_score# ppNo4,6
        elif i < 80:
            score[i,:] = score[i,:] - optim_thresh + adjust_score2# ppNo4,6
        else:
            score[i,:] = score[i,:] - optim_thresh + adjust_score3# ppNo4,6
            
    # ppNo7
    birdcall_one = -1*np.ones(score.shape)
    birdcall_one[score>0] = 1
    
    for i in range(birdcall_one.shape[1]): # each species
        sum_ = 0
        target_species = birdcall_one[:,i]
        if np.sum(target_species[target_species==1]) >= 6:# speed up
            for j in range(birdcall_one.shape[0]): # each time
                if j == 0: # zero time
                    if birdcall_one[j,i] == 1:
                        begin = j
                        sum_ += 1
                else:
                    if birdcall_one[j-1,i] * birdcall_one[j,i] > 0:#birdcall -> birdcall or nocall -> nocall
                        if birdcall_one[j,i] > 0:#birdcall -> birdcall
                            sum_ += 1
                    else: #birdcall_one[j,i] * birdcall_one[j,i] < 0:
                        if birdcall_one[j,i] < 0:#birdcall -> nocall
                            if sum_ >= 6:
                                score[begin,i] = -1
                                score[begin+1,i] = -1
                                score[j-1,i] = -1
                                score[j,i] = -1
                            sum_ = 0
                        else: #nocall -> birdcall
                            begin = j
                            sum_ += 1
    
    n_birds = (score >= 0).sum(-1)   # Counting birds
    n_birds = np.clip(n_birds, 0, maxpreds)  # keep at most maxpreds birds
    
    labels = [np.argsort(- score[i])[:n_birds[i]] for i in range(len(preds))]  # Getting the n_birds most likely class indices
    
    class_labels = [" ".join([TARGETS[l] for l in label]) for label in labels]  # Getting class names
    
    return class_labels

In [ ]:
def vote(preds, min_votes=3):
    votes = Counter(preds)
    return [c for c, count in votes.items() if count >= min_votes]

In [ ]:
def reformat_preds(preds, df):
    prediction_df = pd.DataFrame({
        'row_id': df['row_id'].values,
        'birds': preds
    })
    
    prediction_df['birds'] = prediction_df['birds'].replace([''],'nocall')
    
    return prediction_df

In [ ]:
def inference_voting(test_audios,configs, models, optim_thresh, min_votes=3):
    pred_dfs = []
    for audio_path in tqdm(test_audios):
        clip, _ = sf.read(audio_path)
            
        # 1clip分のdfを作成
        seconds = []
        row_ids = []
        for second in range(5, 605, 5):
            row_id = "_".join(audio_path.name.split("_")[:2]) + f"_{second}"
            seconds.append(second)
            row_ids.append(row_id)

        #ppNo2
        site = "ALL"
        if "SSW" in row_ids[0]:
            site = "SSW"
            target_list = np.copy(SSW_list)
        elif "COR" in row_ids[0]:
            site = "COR"
            target_list = np.copy(COR_list)
        elif "COL" in row_ids[0]:
            site = "COL"
            target_list = np.copy(COL_list)
        else:
            target_list = np.copy(SNE_list)
        
        test_df = pd.DataFrame({
            "row_id": row_ids,
            "seconds": seconds
        })
        all_preds = []
        for i in range(len(models)): # モデルの種類
            preds =[]
            for model in models[i]: # fold分
                pred = prediction_for_clip(test_df, clip, model)
                #120行*397クラスの予測が帰ってくる
                preds.append(pred)
            preds = np.mean(preds, 0)
            
            # postprocess
            #preds_pp = post_process_site_12(preds, target_list, optim_thresh[i])
            if (site == "COR") or (site == "SSW"):
                preds_pp = post_process_site_12(preds, target_list, config.TEYO.models[i]["optim_thresh"][site][0])
            elif (site == "COL"):
                preds_pp = post_process_site_12(preds, target_list, config.TEYO.models[i]["optim_thresh"]["COR"][0])
            else:
                preds_pp = post_process_site_12(preds, target_list, config.TEYO.models[i]["optim_thresh"]["ALL"][0])
            all_preds.append(preds_pp)
            #print("Predicted classes :", preds_pp)
            
        final_preds = []
        #print(f"{len(all_preds[0])},{len(all_preds)}")
        for i in range(len(all_preds[0])): # 120
            preds = []
            for m in range(len(all_preds)): # モデル種数
                preds += all_preds[m][i].split(' ')
                
            final_pred = vote(preds, min_votes=min_votes)
            final_preds.append(' '.join(final_pred))
        #print("\n    -> Voted classes :", final_preds)
        pred_df = reformat_preds(final_preds, test_df)
        pred_dfs.append(pred_df)
    
    sub = pd.concat(pred_dfs, axis=0, sort=False).reset_index(drop=True)
    return sub

In [ ]:
class config:
    INPUT_ROOT = "/kaggle/input/birdclef-2021"
    WORK_ROOT = "/kaggle/working"
    SAMPLE_RATE = 32000
    SEED = 416
    SITE_LST = ["COL", "COR", "SNE", "SSW"]
    N_LABELS = 397
        
    class TEYO:
        n_mels = 128
        fmin = 20
        fmax = 16000
        n_fft = 2048
        hop_length = 512
        models = [
            {"name": 'rexnet_200',
             "weights": [f'../input/bird2021/exp13_rexnet200_last/rexnet_200_fold{i}_last.pth' for i in range(5)],
             "spec_aggreagation":'deltas',
             "optim_thresh": "../input/bird2021/exp13_rexnet200_last/ex13_rexnet_last_optim_thresh.pkl"},
            
            {"name": 'rexnet_200',
             "weights": [f'../input/bird2021/exp17_rexnet_200_loss/rexnet_200_fold{i}_best_loss.pth' for i in range(5)],
             "spec_aggreagation":'deltas',
             "optim_thresh":  "../input/bird2021/exp17_rexnet_200_loss/ex17_rexnet_loss_optim_thresh.pkl"},
            
            {"name": 'rexnet_200',
             "weights": [f'../input/bird2021/exp23_rexnet200/rexnet_200_fold{i}_best_f1.pth' for i in range(2)],
             "spec_aggreagation":'deltas',
             "optim_thresh":  "../input/bird2021/exp23_rexnet200/ex23_rexnet_f1_optim_thresh.pkl"},
            
            {"name": 'densenet161',
             "weights": [f'../input/bird2021/exp23_dense161_f1/densenet161_fold{i}_best_f1.pth' for i in range(5)],
             "spec_aggreagation":'deltas',
             "optim_thresh" : "../input/bird2021/exp23_dense161_loss/ex23_dense161_loss_optim_thresh.pkl"},
    # 
            # {"name": 'tf_efficientnetv2_s',
            #  "weights": [f'../input/bird2021/exp19_effnetv2f1/tf_efficientnetv2_s_fold{i}_best_f1.pth' for i in range(5)],
            #  "spec_aggreagation":'deltas',
            #  "optim_thresh": "../input/bird2021/exp19_effnetv2f1/ex19_effv2_f1_optim_thresh.pkl"},
            # 
            # {"name": 'resnext50_32x4d',
            #  "weights": [f'../input/bird2021/exp17_resnext_f1/resnext50_32x4d_fold{i}_best_f1.pth' for i in range(4)],
            #  "spec_aggreagation":'deltas',
            #  "optim_thresh" : "../input/bird2021/exp17_resnext_f1/ex17_resnext_f1_optim_thresh.pkl"},
    # 
            {"name": 'resnest50d',
             "weights": [f'../input/bird2021/exp17_resnest_f1/resnest50d_fold{i}_best_f1.pth' for i in range(4)],
             "spec_aggreagation":'deltas',
             "optim_thresh": "../input/bird2021/exp17_resnest_f1/ex17_resnest_f1_optim_thresh.pkl"},
        ]
import cloudpickle
for i in range(len(config.TEYO.models)):
    path = config.TEYO.models[i]["optim_thresh"]
    with open(path, "rb") as f:
        config.TEYO.models[i]["optim_thresh"] = cloudpickle.loads(f.read())
    for site in ["SSW", "COR", "ALL"]:
        config.TEYO.models[i]["optim_thresh"][site][0][265] = config.TEYO.models[i]["optim_thresh"][site][0][265] - 0.4#reevir1
        config.TEYO.models[i]["optim_thresh"][site][0][286] = config.TEYO.models[i]["optim_thresh"][site][0][286] - 0.2#rucwar
        config.TEYO.models[i]["optim_thresh"][site][0][315] = config.TEYO.models[i]["optim_thresh"][site][0][315] - 0.4#sonspa
        config.TEYO.models[i]["optim_thresh"][site][0][130] = config.TEYO.models[i]["optim_thresh"][site][0][130] - 0.2#eawpew
        config.TEYO.models[i]["optim_thresh"][site][0][168] = config.TEYO.models[i]["optim_thresh"][site][0][168] - 0.2#grycat
        config.TEYO.models[i]["optim_thresh"][site][0][357] = config.TEYO.models[i]["optim_thresh"][site][0][357] - 0.2#westan
        config.TEYO.models[i]["optim_thresh"][site][0][148] = config.TEYO.models[i]["optim_thresh"][site][0][148] - 0.2#gockin
        config.TEYO.models[i]["optim_thresh"][site][0][216] = config.TEYO.models[i]["optim_thresh"][site][0][216] - 0.2#mouqua
        config.TEYO.models[i]["optim_thresh"][site][0][213] = config.TEYO.models[i]["optim_thresh"][site][0][213] - 0.2#mouchi
        config.TEYO.models[i]["optim_thresh"][site][0][58]  = config.TEYO.models[i]["optim_thresh"][site][0][58] - 0.2#brncre

In [ ]:
models = []
for _conf in config.TEYO.models:
    print(_conf['name'])
    models_ = []
    for weights in _conf['weights']:
        model = get_model(model_name=_conf['name'], spec_aggreagation=_conf['spec_aggreagation'])
        load_model_weights(model, weights)
        model.eval()
        models_.append(model)
    models.append(models_)

# prediction

In [ ]:
optim_thresh = np.load("../input/birdclef-infer-ppno2-3-4-6-7-rex200s-serchth/optim_thresh.npy")

In [ ]:
len(models)

In [ ]:
min_votes = 3
submission = inference_voting(test_audios=all_audios,
                              configs=config,
                              models=models,
                              optim_thresh = optim_thresh,
                              min_votes=min_votes)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)
# submission

In [ ]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}
    

sub = pd.read_csv("./submission.csv")
if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(sub, how="left", on="row_id")
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

In [ ]:
np.save("optim_thresh.npy", optim_thresh)

##### 